In [10]:
#import dependencies
import requests 
import pandas as pd
import numpy as np
import math
import re 
import json

In [11]:
#define max records per page based on API documentation
#define lats and lngs for the gta to be used as paramaters in the post request
#define number of data points being requested from the api
#select records per page to be used as paramater in post request (max records or user defined record if it is lower than max record)
#define number of pages needed to get the requested number of data points (user define record count divided by max records per page)

MAX_RECORDS = 200
latmin = 43.6406
latmax = 43.736439
lngmax = -79.252357       
lngmin = -79.549301
records = 3000
records_per_page = min(records, MAX_RECORDS)
pages = math.ceil(records / MAX_RECORDS)

#create dataframe

maindf = pd.DataFrame({"Address":''},index=[np.arange(0, records)])
maindf["Sale Price"]= ""    
maindf['Mls Number']=""
maindf['Near By Ammenities']= ""
maindf["Ownership Type"] = ""
maindf["Parking"] = ""
maindf["Bathrooms"] = ""
maindf["Bedrooms"] = ""

#loop through each page of the api
#define api url
#set api paramaters
#create a post request to obtain api response
#loop through response results and unpack tuple using enumerate to set index value and loop through each realtor posting found on one page
#change index value based on page number to ensure dataframe rows are not overwritten on each additional page
#add data points to dataframe if lng and lat are within defined area and are not already in the dataframe

for page in np.arange(pages):
    url = 'https://api2.realtor.ca/Listing.svc/PropertySearch_Post'
    body = {
        "CultureId":1,
        "ApplicationId":1,
        "PropertySearchTypeId":1,
        "TransactionTypeID":2,
        "BuildingTypeId":0,
        "LongitudeMin":f'{lngmin}',
        "LongitudeMax":f'{lngmax}',
        "LatitudeMin":f'{latmin}',
        "LatitudeMax":f'{latmax}',
        "AirCondition":1,
        "PriceMin":250000,
        "RecordsPerPage": records_per_page,
        "CurrentPage": page + 1
    }

    response = requests.post(url, data=body).json() 
    
    for index, mls_post in enumerate(response['Results']):
        if page > 0:
            index += (MAX_RECORDS * page)
            
        try:                        
            lng = float(mls_post['Property']['Address']['Longitude'])
            address = mls_post['Property']['Address']
    
            if (lng >= lngmin and lng <= lngmax and address['AddressText'] not in maindf['Address'].unique()):
                maindf.loc[index,"Address"] = address['AddressText']        
                maindf.loc[index,"Mls Number"]= mls_post['MlsNumber']
                maindf.loc[index,"Date Updated"] = mls_post['Individual'][0]['Organization']['PhotoLastupdate']
                    
                try:
                    maindf.loc[index,"Sale Price"] = mls_post['Property']['Price']
                    maindf.loc[index,"Near By Ammenities"] = mls_post['Property']['AmmenitiesNearBy']
                    maindf.loc[index,"Ownership Type"] = mls_post['Property']['OwnershipType']
                    maindf.loc[index,"Parking"] = mls_post['Property']['Parking'][0]['Name']
                    maindf.loc[index,"Bathrooms"] = mls_post['Building']['BathroomTotal']
                    maindf.loc[index,"Bedrooms"] = mls_post['Building']['Bedrooms']
                except:
                    maindf.loc[index,"Sale Price"] = np.nan
                    maindf.loc[index,"Near By Ammenities"] = np.nan
                    maindf.loc[index,"Ownership Type"] = np.nan
                    maindf.loc[index,"Parking"] = np.nan
                    maindf.loc[index,"Bathrooms"] = np.nan
                    maindf.loc[index,"Bedrooms"] = np.nan
                
        except Exception as e:
            print('Error ', e)  
            
maindfclean = maindf.loc[maindf['Address']!="",:]

In [12]:
#drop null values
maindfclean = maindf.dropna(how='any')

In [13]:
#set mls number as index 
maindfclean.set_index('Mls Number')
maindfclean.head()

Address Sale Price Mls Number  \
0  #628 -2737 KEELE ST|Toronto, Ontario M3M2E9   $283,900   W4652611   
1  #614 -2737 KEELE ST|Toronto, Ontario M3M2E9   $294,500   W4653141   
4  #910 -2737 KEELE ST|Toronto, Ontario M3M2E9   $339,700   W4641655   
5  #907 -234 ALBION RD|Toronto, Ontario M9W6A5   $355,000   W4622564   
7  #111 -234 ALBION RD|Toronto, Ontario M9W6A5   $379,999   W4618026   

                        Near By Ammenities      Ownership Type      Parking  \
0  Hospital, Park, Public Transit, Schools  Condominium/Strata  Underground   
1                 Hospital, Public Transit  Condominium/Strata  Underground   
4  Hospital, Park, Public Transit, Schools  Condominium/Strata  Underground   
5                  Hospital, Park, Schools  Condominium/Strata  Underground   
7        Hospital, Public Transit, Schools  Condominium/Strata  Underground   

  Bathrooms Bedrooms           Date Updated  
0         1        1  2019-12-15 2:24:42 PM  
1         1        1  2019-12-13 6:36:20 PM  
4         1        2  2019-12-17 2:28:08 PM  
5         1        2  2019-12-09 8:24:39 AM  
7         2        3  2019-12-09 2:08:48 PM

In [14]:
pattern = re.compile(r'(\d\d\d\d-\d\d-\d\d)')

In [15]:
#use regex to remove time from date updated column
maindfclean['Date Updated'] = maindfclean['Date Updated'].str.extract(pattern)


C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
maindfclean.head()

Address Sale Price Mls Number  \
0  #628 -2737 KEELE ST|Toronto, Ontario M3M2E9   $283,900   W4652611   
1  #614 -2737 KEELE ST|Toronto, Ontario M3M2E9   $294,500   W4653141   
4  #910 -2737 KEELE ST|Toronto, Ontario M3M2E9   $339,700   W4641655   
5  #907 -234 ALBION RD|Toronto, Ontario M9W6A5   $355,000   W4622564   
7  #111 -234 ALBION RD|Toronto, Ontario M9W6A5   $379,999   W4618026   

                        Near By Ammenities      Ownership Type      Parking  \
0  Hospital, Park, Public Transit, Schools  Condominium/Strata  Underground   
1                 Hospital, Public Transit  Condominium/Strata  Underground   
4  Hospital, Park, Public Transit, Schools  Condominium/Strata  Underground   
5                  Hospital, Park, Schools  Condominium/Strata  Underground   
7        Hospital, Public Transit, Schools  Condominium/Strata  Underground   

  Bathrooms Bedrooms Date Updated  
0         1        1   2019-12-15  
1         1        1   2019-12-13  
4         1        2   2019-12-17  
5         1        2   2019-12-09  
7         2        3   2019-12-09

In [17]:
#remove dollar sign from sale price column
maindfclean['Sale Price'] = maindfclean['Sale Price'].str[1:]
maindfclean.head()

C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Address Sale Price Mls Number  \
0  #628 -2737 KEELE ST|Toronto, Ontario M3M2E9    283,900   W4652611   
1  #614 -2737 KEELE ST|Toronto, Ontario M3M2E9    294,500   W4653141   
4  #910 -2737 KEELE ST|Toronto, Ontario M3M2E9    339,700   W4641655   
5  #907 -234 ALBION RD|Toronto, Ontario M9W6A5    355,000   W4622564   
7  #111 -234 ALBION RD|Toronto, Ontario M9W6A5    379,999   W4618026   

                        Near By Ammenities      Ownership Type      Parking  \
0  Hospital, Park, Public Transit, Schools  Condominium/Strata  Underground   
1                 Hospital, Public Transit  Condominium/Strata  Underground   
4  Hospital, Park, Public Transit, Schools  Condominium/Strata  Underground   
5                  Hospital, Park, Schools  Condominium/Strata  Underground   
7        Hospital, Public Transit, Schools  Condominium/Strata  Underground   

  Bathrooms Bedrooms Date Updated  
0         1        1   2019-12-15  
1         1        1   2019-12-13  
4         1        2   2019-12-17  
5         1        2   2019-12-09  
7         2        3   2019-12-09

In [18]:
maindfclean.count()

Address               771
Sale Price            771
Mls Number            771
Near By Ammenities    771
Ownership Type        771
Parking               771
Bathrooms             771
Bedrooms              771
Date Updated          771
dtype: int64